In [1]:
!pip install jupyter_ui_poll

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from IPython.display import display, clear_output, HTML, Image
import random

import time
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

import os
import re

In [3]:
def display_img(img_file):
    style_str = f'width: 500px;'
    html_out = HTML(f"<img style='{style_str}' src={img_file}></img>")
    display(html_out)

In [4]:
# Create Time-out
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [5]:
def ANS_test_single(img_file, blank_img, right_answer):

    top_area = widgets.Output(layout={"height":"60px"})
    main_area = widgets.Output(layout={"height":"350px"})
    bottom_area = widgets.Output(layout={"height":"60px"})

    btn1 = widgets.Button(description="Left")
    btn2 = widgets.Button(description="Right")

    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    panel = widgets.HBox([btn1, btn2])

    top_area.append_display_data( HTML("<h1>Which side has more dots?</h1>") )
    bottom_area.append_display_data(panel)

    display(top_area)
    display(main_area)
    display(bottom_area)

    with main_area: 
        display_img(img_file) # Display the image
        time.sleep(0.75) # Wait for 0.75 seconds

        clear_output(wait=True)
        display_img(blank_img)  # Display a blank image or placeholder imag
    
    result = wait_for_event(timeout=3)
    
    score = 0
    
    if result['description'] == '':
        with main_area: display("Too slow!")
    elif result['description'] ==  right_answer:
        with main_area: score = 1
    else:
        with main_area: score = 0

    wait_for_event(timeout=1.5)

    clear_output()
        
    return score, result['description']

In [6]:
# Read file name and create a dictionary that contains 
# the image name as key and 
# the answer (Right or Left) as corresponding value


# Get the current working directory
current_directory = os.getcwd()

# Specify the relative path to local files
relative_path = ''

# Combine the current directory and relative path to get the full path
directory_path = os.path.join(current_directory, relative_path)


file_dict = {}

# Define a regular expression pattern to extract numbers from the file names
pattern = re.compile(r'dots_image_(\d{1,2})x(\d{1,2}).png')

# Iterate over files in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a PNG file
    if filename.endswith('.png'):
        # Use regular expression to extract numbers from the file name
        match = pattern.match(filename)
        if match:
            left_side_num = int(match.group(1))
            right_side_num = int(match.group(2))

            # Compare left_side_num and right_side_num
            if left_side_num > right_side_num:
                file_dict[filename] = 'Left'
            elif left_side_num < right_side_num:
                file_dict[filename] = 'Right'
            else:
                file_dict[filename] = 'Unknown'

# # Test
# print(file_dict)

In [7]:
def create_shuffled_list(file_dict):
    
    # Get the image files and answers from file_dict
    image_files = list(file_dict.keys())
    answers = list(file_dict.values())
    
    # Assume the test contained 64 trials
    # As there are 16 images in original files, we need to repeat 4 times
    # total_image_in_block = image_files * 4
    # total_answers_in_block = answers * 4

    # Combine image files and answers into pairs
    pairs = list(zip(image_files, answers))
    # pairs = list(zip(total_image_in_block, total_answers_in_block))

    # Shuffle the pairs
    random.shuffle(pairs)

    # Unzip the pairs back into separate lists
    shuffled_image_files, shuffled_answers = zip(*pairs)

    # Convert the result back to lists
    shuffled_image_files = list(shuffled_image_files)
    shuffled_answers = list(shuffled_answers)

    return shuffled_image_files, shuffled_answers
    
# # Test
# shuffled_image_files, shuffled_answers = create_shuffled_list(file_dict)
# print(shuffled_image_files)
# print(shuffled_answers)

In [8]:
def run_ANS_test_block():
    total = 0
    input_answers = []

    shuffled_image, shuffled_answers = create_shuffled_list(file_dict)
    blank_img = "dots_image_blank.png"

    for image, ans in zip(shuffled_image, shuffled_answers):
        score, input_ans = ANS_test_single(image, blank_img, ans)
        total += score
        
        input_answers.append(input_ans)
        
    print(f'You scored {total} out of 16')
    return total, shuffled_image, shuffled_answers, input_answers


total, shuffled_image, shuffled_answers, input_answers = run_ANS_test_block()
print(total)
print(shuffled_image)
print(shuffled_answers)
print(input_answers)

You scored 10 out of 16
10
['dots_image_9x10.png', 'dots_image_18x16.png', 'dots_image_15x20.png', 'dots_image_16x18.png', 'dots_image_9x12.png', 'dots_image_20x18.png', 'dots_image_10x9.png', 'dots_image_21x18.png', 'dots_image_14x12.png', 'dots_image_18x20.png', 'dots_image_20x15.png', 'dots_image_16x12.png', 'dots_image_12x16.png', 'dots_image_12x14.png', 'dots_image_12x9.png', 'dots_image_18x21.png']
['Right', 'Left', 'Right', 'Right', 'Right', 'Left', 'Left', 'Left', 'Left', 'Right', 'Left', 'Left', 'Right', 'Right', 'Left', 'Right']
['Left', 'Left', '', 'Left', 'Right', 'Left', '', 'Left', 'Left', 'Left', 'Left', 'Left', 'Right', 'Left', 'Left', 'Right']


In [9]:
# Send data to google form

import requests
from bs4 import BeautifulSoup
import json

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

# Let user enter their names, age and major or job
print("We are going to test your reaction time!")

print("Please enter your name:")
ans1 = input("> ")

print("Please enter your age:")
ans2 = input("> ")

print("Please indicate whether your major or occupation is more aligned with science or art:")
ans3 = input("> ")


data_dict = {
    'name': ans1,
    'age': ans2,
    'subject': ans3,
    'score': total,
    'image names': shuffled_image, 
    'image answers': shuffled_answers,
    'input answers': input_answers
}

form_url = 'https://docs.google.com/forms/d/e/1FAIpQLScWXr-igXq9VC9caYLvd4v_vnf--KyrpeLZjAqiyJA0CzcSXg/viewform?usp=sf_link'
send_to_google_form(data_dict, form_url)


We are going to test your reaction time!
Please enter your name:


>  s2


Please enter your age:


>  21


Please indicate whether your major or occupation is more aligned with science or art:


>  sc


True